In [16]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline
import nltk
from textblob import TextBlob
from collections import Counter
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
tweets = pd.read_csv("Tweets 300 en ingles.csv")
tweets

,Fecha,Genero,Commnet
0,02/01/24,Hombre,My passage through Mexico was complicated.
1,05/16/24,Mujer,"The authorities were kind, but the risks on th..."
2,05/04/24,Mujer,I hope to reach my destination in Mexico; it h...
3,07/10/24,Mujer,"Every day in Mexico is a new challenge, but ea..."
4,01/21/24,Hombre,My passage through Mexico has taught me to be ...
...,...,...,...
294,07/02/24,Mujer,The solidarity of some Mexicans has been unexp...
295,01/20/24,Hombre,"The days pass slowly and full of challenges, b..."
296,08/15/24,Hombre,"Despite everything, I still believe that I wil..."
297,12/07/24,Mujer,"I have been robbed on the way, but I have not ..."


In [4]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Fecha    299 non-null    object
 1   Genero   299 non-null    object
 2   Commnet  299 non-null    object
dtypes: object(3)
memory usage: 7.1+ KB


In [5]:
tweets['Fecha'] = pd.to_datetime(tweets['Fecha'], format='%m/%d/%y')  # Ejemplo para formato AAAA-MM-DD

In [6]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Fecha    299 non-null    datetime64[ns]
 1   Genero   299 non-null    object        
 2   Commnet  299 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 7.1+ KB


In [7]:
tweets = tweets.rename(columns={'Commnet': 'comment'})

In [8]:
# Crear una función para obtener el sentimiento
def obtener_sentimiento(texto):
    analysis = TextBlob(texto)
    return analysis.sentiment.polarity  # Polaridad de -1 (negativo) a 1 (positivo)

# Crear la nueva columna 'sentimiento_porcentaje' con los valores de polaridad
tweets['porcentaje'] = tweets['comment'].apply(obtener_sentimiento)

tweets

,Fecha,Genero,comment,porcentaje
0,2024-02-01,Hombre,My passage through Mexico was complicated.,-0.500000
1,2024-05-16,Mujer,"The authorities were kind, but the risks on th...",0.380000
2,2024-05-04,Mujer,I hope to reach my destination in Mexico; it h...,0.250000
3,2024-07-10,Mujer,"Every day in Mexico is a new challenge, but ea...",0.136364
4,2024-01-21,Hombre,My passage through Mexico has taught me to be ...,0.433333
...,...,...,...,...
294,2024-07-02,Mujer,The solidarity of some Mexicans has been unexp...,0.150000
295,2024-01-20,Hombre,"The days pass slowly and full of challenges, b...",0.025000
296,2024-08-15,Hombre,"Despite everything, I still believe that I wil...",0.212121
297,2024-12-07,Mujer,"I have been robbed on the way, but I have not ...",0.000000


In [9]:

# Función para obtener el sentimiento en un rango de -1 a 1
def obtener_sentimiento(texto):
    analysis = TextBlob(texto)
    return analysis.sentiment.polarity

# Función para clasificar el sentimiento en categorías
def clasificar_sentimiento(polaridad):
    if 0.6 <= polaridad <= 1:
        return 'Alegría'
    elif 0.3 <= polaridad < 0.6:
        return 'Positivo'
    elif -0.2 <= polaridad < 0.3:
        return 'Neutral'
    elif -0.5 <= polaridad < -0.2:
        return 'Negativo'
    else:
        return 'Tristeza'

# Crear la columna 'porcentaje' con los valores de polaridad
tweets['porcentaje'] = tweets['comment'].apply(obtener_sentimiento)

# Crear la columna 'tipo_sentimiento' con la clasificación
tweets['tipo_sentimiento'] = tweets['porcentaje'].apply(clasificar_sentimiento)

# Verificar los primeros resultados
tweets

,Fecha,Genero,comment,porcentaje,tipo_sentimiento
0,2024-02-01,Hombre,My passage through Mexico was complicated.,-0.500000,Negativo
1,2024-05-16,Mujer,"The authorities were kind, but the risks on th...",0.380000,Positivo
2,2024-05-04,Mujer,I hope to reach my destination in Mexico; it h...,0.250000,Neutral
3,2024-07-10,Mujer,"Every day in Mexico is a new challenge, but ea...",0.136364,Neutral
4,2024-01-21,Hombre,My passage through Mexico has taught me to be ...,0.433333,Positivo
...,...,...,...,...,...
294,2024-07-02,Mujer,The solidarity of some Mexicans has been unexp...,0.150000,Neutral
295,2024-01-20,Hombre,"The days pass slowly and full of challenges, b...",0.025000,Neutral
296,2024-08-15,Hombre,"Despite everything, I still believe that I wil...",0.212121,Neutral
297,2024-12-07,Mujer,"I have been robbed on the way, but I have not ...",0.000000,Neutral


### Proceso de Clasificación
Definición del Rango de la Polaridad: La polaridad del sentimiento se asigna a cada texto (por ejemplo, un tweet) utilizando un valor numérico entre -1 y 1:

- -1 indica el sentimiento más negativo posible.
- 1 representa el sentimiento más positivo posible.
- 0 sugiere un sentimiento neutral o sin carga emocional significativa.
- Transformación a Categorías Cualitativas: La función divide este rango en cinco intervalos predefinidos, donde cada uno corresponde a una categoría emocional específica. El proceso se denomina discretización, y permite simplificar los datos numéricos en etiquetas cualitativas, - facilitando el análisis e interpretación.

### Distribución de los Intervalos: La polaridad se clasifica en los siguientes grupos:

- Alegría: [0.6, 1]
Intervalos con los valores más altos, sugiriendo una fuerte emoción positiva.
- Positivo: [0.3, 0.6)
Polaridades moderadamente positivas.
- Neutral: [-0.2, 0.3)
Comentarios con poca o nula carga emocional, lo que sugiere una distribución cercana al centro.
- Negativo: [-0.5, -0.2)
Sentimientos ligeramente negativos.
- Tristeza: [-1, -0.5)
Polaridades muy bajas, indicando una carga emocional negativa intensa.